### Gruppo 5.a: Castagnacci Giulia, Giordano Elisabetta

### Analisi spaceindex

In [1]:

using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using IntervalTrees
using SparseArrays
using NearestNeighbors
using BenchmarkTools
using OrderedCollections
using Base.Threads


### Dati in input

In [2]:
V = hcat([[0.,0],[1,0],[1,1],[0,1],[2,1]]...);                                          #vertici del modello 2D
V3 = hcat([[0.,0,0],[1,0,3],[1,1,2],[0,1,1],[2,1,0]]...);                               #vertici del modello 3D
EV = [[1,2],[2,3],[3,4],[4,1],[1,5]];                                                   #spigoli del modello
bb = [[0.0 1.0; 0.0 0.0], [1.0 1.0; 0.0 1.0], [0.0 1.0; 1.0 1.0], [0.0 0.0; 0.0 1.0], [0.0 2.0; 0.0 1.0]];  #bounding box
dict = OrderedDict([0.0, 1.0] => [1, 3],[1.0, 1.0] => [2],[0.0, 0.0] => [4],[0.0, 2.0] => [5])  #dizionario intervallo/indice
cov = [[4, 1, 3, 5, 2], [1, 3, 5, 2], [4, 1, 3, 5, 2], [4, 1, 3, 5], [4, 1, 3, 5, 2]]    #intersezioni tra bounding box

5-element Vector{Vector{Int64}}:
 [4, 1, 3, 5, 2]
 [1, 3, 5, 2]
 [4, 1, 3, 5, 2]
 [4, 1, 3, 5]
 [4, 1, 3, 5, 2]

## Funzioni aggiuntive create

### createIntervalTree
Aggiunta funzione createIntervalTree, struttura dati che contiene intervalli e che consente di trovare in modo efficiente tutti gli intervalli che si sovrappongono a un determinato intervallo o punto. Utilizzata in: spaceindex, boxcovering

In [3]:
function createIntervalTree(boxdict::AbstractDict{Array{Float64,1},Array{Int64,1}})
    tree = IntervalTrees.IntervalMap{Float64,Array}()
    for (key, boxset) in boxdict
        tree[tuple(key...)] = boxset
    end
    return tree
end

createIntervalTree (generic function with 1 method)

In [20]:
t = createIntervalTree(dict) #creazione albero

IntervalTrees.IntervalBTree{Float64, IntervalValue{Float64, Array}, 64}


(0.0,0.0) => [4]
(0.0,1.0) => [1, 3]
(0.0,2.0) => [5]


(1.0,1.0) => [2]

### removeIntersection
Funzione che elimina le intersezioni di ogni bounding box con loro stessi. Utilizzata in: spaceindex

In [4]:
function removeIntersection(covers::Array{Array{Int64,1},1})
    @threads for k=1:length(covers)
        covers[k] = setdiff(covers[k],[k])	#toglie le intersezioni con se stesso 
    end
end

removeIntersection (generic function with 1 method)

### addIntersection
Funzione che aggiunge gli elementi di iterator nell'i-esimo array di covers. Utilizzata in: spaceindex, boxcovering

In [5]:
function addIntersection(covers::Array{Array{Int64,1},1}, i::Int64, iterator)
    splice!(covers[i],1)		#splice serve a togliere gli zeri iniziali all'interno di covers
    @threads for x in collect(iterator)
        append!(covers[i],x.value)
    end
end

addIntersection (generic function with 1 method)

## Funzioni di supporto

### boudingbox

In [6]:
function boundingbox2(vertices::Lar.Points)
    firstDim = vertices[1,:]
    secondDim = vertices[2,:]
    if (size(vertices,1)==3)
        thirdDim = vertices[3,:]
         minimum = Threads.@spawn hcat([min(firstDim...), min(secondDim...), min(thirdDim...)])
         maximum = Threads.@spawn hcat([max(firstDim...), max(secondDim...), max(thirdDim...)])
    else
         minimum = Threads.@spawn hcat([min(firstDim...), min(secondDim...)])
         maximum = Threads.@spawn hcat([max(firstDim...), max(secondDim...)])
    end
    return fetch(minimum),fetch(maximum)
 end

boundingbox2 (generic function with 1 method)

### boxcovering

In [7]:
function boxcovering2(bboxes, index, tree)
    covers = [[zero(eltype(Int64))] for k=1:length(bboxes)]		#zero(eltype(Int64)) serve per rendere covers type stable
    @threads for (i,boundingbox) in collect(enumerate(bboxes))
        extent = bboxes[i][index,:]
        iterator = IntervalTrees.intersect(tree, tuple(extent...))
        addIntersection(covers, i, iterator)
    end
    return covers
end

boxcovering2 (generic function with 1 method)

### coordintervals

In [8]:
function coordintervals(coord,bboxes)
	boxdict = OrderedDict{Array{Float64,1},Array{Int64,1}}()
	for (h,box) in enumerate(bboxes)
		key = box[coord,:]
		if haskey(boxdict,key) == false
			boxdict[key] = [h]
		else
			push!(boxdict[key], h)
		end
	end
	return boxdict
end

coordintervals (generic function with 1 method)

### Versione iniziale di spaceindex

Funzione che prende in input una tupla costituita da una matrice che contiene i punti del modello, di tipo Lar.LAR, e da un array di array che contiene le celle cioè scomposizioni dello spazio geometrico. La funzione restituisce in output un array di array dove l'elemento i-esimo rappresenta quali intersezioni ha il bounding box i-esimo con gli altri bounding box. Nello specifico, la funzione calcola le 1-celle e il loro bounding box attraverso la funzione boundingBox. Si suddividono le coordinate x e y in due dizionari chiamando la funzione coordintervals. Per entrambe le coordinate x e y, si calcola un intervalTree cioè una struttura dati che contiene intervalli. La funzione boxCovering viene chiamata per calcolare le sovrapposizioni sulle singole dimensioni dei bounding Box. Intersecando quest'ultime, si ottengono le intersezioni effettive tra bounding box. La funzione esegue lo stesso procedimento sulla coordinata z se presente. Infine, si eliminano le intersezioni di ogni bounding box con loro stessi.

In [9]:
function spaceindex(model::Lar.LAR)::Array{Array{Int,1},1}
	V,CV = model[1:2]
	dim = size(V,1)
	cellpoints = [ V[:,CV[k]]::Lar.Points for k=1:length(CV) ]
	#----------------------------------------------------------
	bboxes = [hcat(Lar.boundingbox(cell)...) for cell in cellpoints]
	xboxdict = Lar.coordintervals(1,bboxes)
	yboxdict = Lar.coordintervals(2,bboxes)
	# xs,ys are IntervalTree type
	xs = IntervalTrees.IntervalMap{Float64, Array}()
	for (key,boxset) in xboxdict
		xs[tuple(key...)] = boxset
	end
	ys = IntervalTrees.IntervalMap{Float64, Array}()
	for (key,boxset) in yboxdict
		ys[tuple(key...)] = boxset
	end
	xcovers = Lar.boxcovering(bboxes, 1, xs)
	ycovers = Lar.boxcovering(bboxes, 2, ys)
	covers = [intersect(pair...) for pair in zip(xcovers,ycovers)]

	if dim == 3
		zboxdict = Lar.coordintervals(3,bboxes)
		zs = IntervalTrees.IntervalMap{Float64, Array}()
		for (key,boxset) in zboxdict
			zs[tuple(key...)] = boxset
		end
		zcovers = Lar.boxcovering(bboxes, 3, zs)
		covers = [intersect(pair...) for pair in zip(zcovers,covers)]
	end
	# remove each cell from its cover
	for k=1:length(covers)
		covers[k] = setdiff(covers[k],[k])
	end
	return covers
end

spaceindex (generic function with 1 method)

### Analisi del comportamento e dei tempi della versione iniziale

In [10]:
@btime spaceindex((V,EV))

  140.600 μs (1107 allocations: 49.16 KiB)


5-element Vector{Vector{Int64}}:
 [4, 5, 2]
 [1, 3, 5]
 [4, 5, 2]
 [1, 3, 5]
 [4, 1, 3, 2]

In [11]:
@code_warntype spaceindex((V,EV))

MethodInstance for spaceindex(::

Tuple{Matrix{Float64}, Vector{Vector{Int64}}})


  from spaceindex(model::Union{Tuple{Matrix, Vector{Vector{Int64}}}, Tuple{Matrix, Vector{Vector{Int64}}, Vector{Vector{Int64}}}}) in Main at c:\Users\giord\eclipse-SIW\LARSplitting2D\notebooks\refactoringAnalisi_spaceindex.ipynb:1
Arguments
  

#self#::Core.Const(spaceindex)
  model::Tuple{Matrix{Float64}, Vector{Vector{Int64}}}
Locals
  @_3::Any
  #25::var"#25#29"
  @_5::Any
  #24::var"#24#28"
  @_7::Any
  @_8::Any
  #23::var"#23#27"
  #22::var"#22#26"{Vector{Vector{Int64}}, Matrix{Float64}}
  @_11::Int64
  covers::Any
  zcovers::Any
  zs::IntervalTrees.IntervalBTree{Float64, IntervalValue{Float64, Array}, 64}
  zboxdict::Any
  ycovers::Any
  xcovers::Any
  ys::IntervalTrees.IntervalBTree{Float64, IntervalValue{Float64, Array}, 64}
  xs::IntervalTrees.IntervalBTree{Float64, IntervalValue{Float64, Array}, 64}
  yboxdict::Any
  xboxdict::Any
  bboxes::Vector
  cellpoints::Vector{Matrix{Float64}}
  dim::Int64
  CV::Vector{Vector{Int64}}
  V::Matrix{Float64}
  @_27::Any
  boxset@_28::Any
  key@_29::Any
  @_30::Any
  boxset@_31::Any
  key@_32::Any
  @_33::Any
  boxset@_34::Any
  key@_35::Any
  k::Any
Body::Vector{Vector{Int64}}


1 ── 

...

getfield(%117, 1))
│           (@_33 = Core.getfield(%117, 2))
│    %120 = Base.indexed_iterate(%116, 2, @_33)::Any
│           (boxset@_34 = Core.getfield(%120, 1))
│    %122 = Core.getfield(%115, 2)::Any
│    %123 = zs::IntervalTrees.IntervalBTree{Float64, IntervalValue{Float64, Array}, 64}
│    %124 = boxset@_34::Any
│    %125 = Core._apply_iterate(Base.iterate, Main.tuple, key@_35)::Tuple
│           Base.setindex!(%123, %124, %125)


│           (@_5 = Base.iterate(%110, %122))
│    %128 = (@_5 === nothing)::Bool
│    %129 = Base.not_int(%128)::Bool
└───        goto #11 if not %129
10 ─        goto #9
11 ┄ %132 = Base.getproperty(Main.Lar, :boxcovering)::Any
│    %133 = bboxes::Vector
│           (zcovers = (%132)(%133, 3, zs))
│           (#25 = %new(Main.:(var"#25#29")))
│    %136 = #25::Core.Const(var"#25#29"())
│   

 %137 = Main.zip(zcovers, covers)::Union{DataStructures.ZippedSparseIntSetIterator{Tuple{DataStructures.SparseIntSet, DataStructures.SparseIntSet}, Tuple{}}, Base.Iterators.Zip}
│    %138 = Base.Generator(%136, %137)::Base.Generator{_A, var"#25#29"} where _A
└───        (covers = Base.collect(%138))
12 ┄ %140 = Main.length(covers)::Any
│    %141 = (1:%140)::Any
│           (@_3 = Base.iterate(%141))
│    %143 = 

(@_3 === nothing)::Bool
│    %144 = Base.not_int(%143)::Bool
└───        goto #15 if not %144
13 ┄ %146 = @_3::Any
│           (k = Core.getfield(%146, 1))
│    %148 = Core.getfield(%146, 2)::Any
│    %149 = Base.getindex(covers, k)::Any
│    %150 = Base.vect(k)::Vector
│    %151 = Main.setdiff(%149, %150)::Any
│           Base.setindex!(covers, %151, k)
│           (@_3 = Base.iterate(%141, %148))
│    %154 = (@_3 === nothing)::Bool
│    %155 = Base.not_int(%154)::Bool
└───        goto #15 if not %155
14 ─        goto #13
15 ┄ %158 = Base.convert(%14, covers)::Any
│    %159 = Core.typeassert(%158, %14)::Vector{Vector{Int64}}
└─── 

       return %159



In [12]:
@benchmark spaceindex((V,EV))

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  141.200 μs …  13.216 ms  ┊ GC (min … max): 0.00% … 96.75%
 Time  (median):     151.550 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   187.658 μs ± 322.743 μs  ┊ GC (mean ± σ):  5.31% ±  3.09%

  ▇█▆▅▄▃▂▃▃▄▂▄▄▂     ▂▆▅▃▂▁                                     ▂
  ████████████████▇▇▇████████▇███▇█▇▇▇▇▆▆▅▅▅▅▅▄▅▅▃▄▄▄▄▃▄▃▃▁▄▄▁▇ █
  141 μs        Histogram: log(frequency) by time        415 μs <

 Memory estimate: 49.16 KiB, allocs estimate: 1107.

### Versione parallelizzata di spaceindex
 Per la parallalizzazione abbiamo usato la macro @spawn e creato delle funzioni aggiuntive di supporto, createIntervalTree, addIntersection (già citate in boxcovering) e removeIntersection, funzione che elimina le intersezioni di ogni bounding box con loro stessi. Grazie a questo, le prestazioni hanno subito un notevole miglioramento

In [13]:
function spaceindex2(model::Lar.LAR)::Array{Array{Int,1},1}
	V,CV = model[1:2]
    dim = size(V,1)
    
    cellpoints = [ V[:,CV[k]]::Lar.Points for k=1:length(CV) ]		    #calcola le celle
    bboxes = [hcat(boundingbox2(cell)...) for cell in cellpoints]    #calcola i boundingbox delle celle
    
    xboxdict = Threads.@spawn coordintervals(1,bboxes)
    yboxdict = Threads.@spawn coordintervals(2,bboxes)

    # xs,ys sono di tipo IntervalTree
    xs = Threads.@spawn createIntervalTree(fetch(xboxdict))
    ys = Threads.@spawn createIntervalTree(fetch(yboxdict))
    
    xcovers = Threads.@spawn boxcovering2(bboxes, 1, fetch(xs))                        #lista delle intersezioni dei bb sulla coordinata x
    ycovers = Threads.@spawn boxcovering2(bboxes, 2, fetch(ys))                        #lista delle intersezioni dei bb sulla coordinata x
    covers = [intersect(pair...) for pair in zip(fetch(xcovers),fetch(ycovers))]      #lista delle intersezioni dei bb su entrambe le coordinate

    if dim == 3
        zboxdict = Threads.@spawn coordintervals(3,bboxes)
        zs = Threads.@spawn createIntervalTree(fetch(zboxdict))
        zcovers = Threads.@spawn boxcovering2(bboxes, 3, fetch(zs))
        covers = [intersect(pair...) for pair in zip(fetch(zcovers),covers)]
    end
    
    removeIntersection(covers)       #rimozione delle intersezioni con se stesso
    return covers
end

spaceindex2 (generic function with 1 method)

### Analisi del comportamento e dei tempi della versione parallelizzata

In [14]:
@btime spaceindex2((V,EV))

  77.100 μs (538 allocations: 40.08 KiB)


5-element Vector{Vector{Int64}}:
 [4, 5, 2]
 [1, 3, 5]
 [4, 5, 2]
 [1, 3, 5]
 [4, 1, 3, 2]

In [15]:
@benchmark spaceindex2((V,EV))

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   87.400 μs …  12.921 ms  ┊ GC (min … max): 0.00% … 98.18%
 Time  (median):     121.000 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   131.105 μs ± 279.528 μs  ┊ GC (mean ± σ):  5.88% ±  2.78%

   ▄                    ▁█▁                                      
  ██▇▅▄▅▅▄▃▂▃▂▂▅▄▄▃▂▂▁▁▁███▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  87.4 μs          Histogram: frequency by time          236 μs <

 Memory estimate: 40.08 KiB, allocs estimate: 538.

## Test

In [16]:
using Test

@testset "spaceindex tests" begin
    V,(VV,EV,FV) = Lar.cuboidGrid([2,1],true)
    EV = [[1, 2], [3, 4], [5, 6], [1, 3], [2, 4], [3, 5], [4, 6]]
    cellpoints = [ V[:,EV[k]]::Lar.Points for k=1:length(EV) ]
    bboxes = [hcat(Lar.boundingbox(cell)...) for cell in cellpoints]
    xboxdict = Lar.coordintervals(1,bboxes)
    yboxdict = Lar.coordintervals(2,bboxes)
    xs = IntervalTrees.IntervalMap{Float64, Array}()
    for (key,boxset) in xboxdict
        xs[tuple(key...)] = boxset
    end
    ys = IntervalTrees.IntervalMap{Float64, Array}()
    for (key,boxset) in yboxdict
        ys[tuple(key...)] = boxset
    end
    xcovers = Lar.boxcovering(bboxes, 1, xs)
    ycovers = Lar.boxcovering(bboxes, 2, ys)
    covers = [intersect(pair...) for pair in zip(xcovers,ycovers)]
    
    @test covers == Array{Int64,1}[[1, 4, 5], [4, 5, 2, 6, 7], [6, 7, 3], 
        [1, 4, 2, 6], [1, 5, 2, 7], [4, 2, 6, 3], [5, 2, 7, 3]]
end

Test Summary:    | Pass  Total


spaceindex tests |    1      1


Test.DefaultTestSet("spaceindex tests", Any[], 1, false, false)

![](https://github.com/GiuliaCastagnacci/LARSplitting2D/blob/main/docs/plot/images/test/tests_spaceindex.png?raw=true)

![](https://github.com/GiuliaCastagnacci/LARSplitting2D/blob/main/docs/plot/screenTest/tests_spaceindex.png?raw=true)

## Test Funzioni aggiuntive create

In [17]:
@testset "createIntervalTree test" begin
    dict = OrderedDict([0.0, 1.0] => [1, 3],[1.0, 1.0] => [2],[0.0, 0.0] => [4],[0.0, 2.0] => [5])
    t = createIntervalTree(dict)
    @test typeof(t) == IntervalTrees.IntervalBTree{Float64,IntervalValue{Float64,Array},64}
    @test t.root.entries[1].first == 0.0
    @test t.root.entries[1].last == 0.0
    @test t.root.entries[1].value == [4]

    @test t.root.entries[2].first == 0.0
    @test t.root.entries[2].last == 1.0
    @test t.root.entries[2].value == [1,3]

    @test t.root.entries[3].first == 0.0
    @test t.root.entries[3].last == 2.0
    @test t.root.entries[3].value == [5]

    @test t.root.entries[4].first == 1.0
    @test t.root.entries[4].last == 1.0
    @test t.root.entries[4].value == [2]
end

Test Summary:           | 

Pass  Total
createIntervalTree test |   13     13


Test.DefaultTestSet("createIntervalTree test", Any[], 13, false, false)

![](https://github.com/GiuliaCastagnacci/LARSplitting2D/blob/main/docs/plot/screenTest/tests_createIntervalTree.png?raw=true)

![](https://github.com/GiuliaCastagnacci/LARSplitting2D/blob/main/docs/plot/images/test/tests_createIntervalTree.png?raw=true)

In [18]:
@testset "removeIntersection test" begin
    covers = [[4, 1, 3, 5, 2], [1, 3, 5, 2], [4, 1, 3, 5, 2], [4, 1, 3, 5], [4, 1, 3, 5, 2]]
    removeIntersection(covers)
    @test typeof(covers) == Array{Array{Int64,1},1}
    @test covers[1] == [4, 3, 5, 2]
    @test covers[2] == [1, 3, 5]
    @test covers[3] == [4, 1, 5, 2]
    @test covers[4] == [1, 3, 5]
    @test covers[5] == [4, 1, 3, 2]

end

Test Summary:           | Pass  Total
removeIntersection test |    6      6


Test.DefaultTestSet("removeIntersection test", Any[], 6, false, false)

![](https://github.com/GiuliaCastagnacci/LARSplitting2D/blob/main/docs/plot/screenTest/tests_removeIntersection.png?raw=true)

![](https://github.com/GiuliaCastagnacci/LARSplitting2D/blob/main/docs/plot/images/test/tests_removeIntersection.png?raw=true)


In [22]:
@testset "addIntersection test" begin
    bb = [[0.0 1.0; 0.0 0.0], [1.0 1.0; 0.0 1.0], [0.0 1.0; 1.0 1.0], [0.0 0.0; 0.0 1.0], [0.0 2.0; 0.0 1.0]];
    dict = OrderedDict([0.0, 1.0] => [1, 3],[1.0, 1.0] => [2],[0.0, 0.0] => [4],[0.0, 2.0] => [5])
    t = createIntervalTree(dict)
    c = boxcovering2(bb,1,t)
    @test c == [[4, 1, 3, 5, 2], [1, 3, 5, 2], [4, 1, 3, 5, 2], [4, 1, 3, 5], [4, 1, 3, 5, 2]];
end

Test Summary:        | Pass  Total
addIntersection test |    1      1


Test.DefaultTestSet("addIntersection test", Any[], 1, false, false)

![](https://github.com/GiuliaCastagnacci/LARSplitting2D/blob/main/docs/plot/screenTest/tests_addIntersection.png?raw=true)

![](https://github.com/GiuliaCastagnacci/LARSplitting2D/blob/main/docs/plot/images/test/tests_addIntersection.png?raw=true)